<a href="https://colab.research.google.com/github/jwkanggist/EverybodyTensorflow2.0/blob/master/lab2_linefitting_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB2 Line-fitting
목적: 가장 기본적인 머신러닝 연습문제인 line fitting 문제를 tf2.0을 가지고 풀어보자!

In [0]:
# preprocessor parts

from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

import numpy as np

from matplotlib import cm
from matplotlib import gridspec
import matplotlib.pyplot as plt

from datetime import datetime

In [0]:
  # for Tensorboard use
  LOG_DIR = 'drive/data/tb_logs'
	
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
	
  import os
  if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
	  
  get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
      .format(LOG_DIR))
	
  get_ipython().system_raw('./ngrok http 6006 &')
	
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# Data pipeline: generation of data set
total_size = 5000
training_size = 4000
validation_size = 1000
xsize = 20

x_data = np.zeros([xsize, total_size])
a_true = 2
b_true = 0.5

for i in range(total_size):
    x_data[:,i] =  np.linspace(0,10,xsize)

noise_var   = 1.0
noise       = np.sqrt(noise_var) * np.random.randn(xsize,total_size)
y_clean     = a_true * x_data + b_true
y_data      = y_clean + noise

x_train = x_data[:,0:training_size].transpose()
y_train = y_data[:,0:training_size].transpose()

x_test = x_data[:,training_size:-1].transpose()
y_test = y_data[:,training_size:-1].transpose()

In [0]:
# model building

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(xsize, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

tensorboard_callback = TensorBoard(log_dir=LOG_DIR, 
                           histogram_freq=1,
                           write_graph=True,
                           write_images=True)

In [0]:
# model training and evaluation part
training_epochs = 20
model.fit(x_train, y_train, 
          epochs=training_epochs,
          validation_data=(x_test, y_test),
          batch_size=128,
          callbacks=[tensorboard_callback])

model.evaluate(x_test,  y_test, verbose=2)

In [0]:
# prediction 

plt.figure(1,figsize=(10,10))
test_input = x_test[0,:]
test_output = y_test[0,:]
pred_y = model.predict(test_input.reshape([1,xsize]))
plt.plot(test_input,test_output,label='noisy data',color='b',marker='*')
plt.plot(test_input, pred_y.reshape([xsize,]),label='prediction',color='r')
plt.legend()
plt.title('A line fitting example:')
plt.xlabel('X data')
plt.ylabel('Y data')